# Download Poligons

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dd360_data import config_utils
import boto3
import os
import sqlalchemy
import datetime as dt

In [11]:
TODAY = dt.datetime.now()
os.environ['EXECUTION_ENVIRONMENT'] = 'development'
os.environ['AWS_PROFILE'] = 'dd3-development'
os.environ['RUNDATE'] = TODAY.strftime("%Y%m%d")
boto3.setup_default_session(region_name="us-east-1")

config_utils.configure_environment()

In [12]:
def pg_conn(db: str) -> sqlalchemy.engine.base.Engine:
    """
    Returns the PostgreSQL connection string
    """
    response = boto3.client("glue").get_connection(
        Name=db,
    )[
        "Connection"
    ]["ConnectionProperties"]
    url = response["JDBC_CONNECTION_URL"].replace("jdbc:postgresql://", "")
    username = response["USERNAME"]
    password = response["PASSWORD"]
    return sqlalchemy.create_engine(
        f"postgresql://{username}:{password}@{url}",
    )


---
# Data


In [8]:
# download municipality data
gdf_mun = (
    gpd.read_postgis(
        sql="SELECT * FROM geospatial_layers.inegi_municipalities",
        conn=pg_conn("data-geospatial-conn"),
        geom_col="geometry"
    )
    # .to_crs(epsg=4326)
)
gdf_mun.head()

EntityNotFoundException: An error occurred (EntityNotFoundException) when calling the GetConnection operation: Connection not found